In [65]:
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from collections import defaultdict

/Users/AlanYAN/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [54]:
def list_gen():
    root_dir = '256_ObjectCategories/'
    image_dirs = glob.glob(root_dir + '*')
    d = defaultdict(list)
    for idir in image_dirs:
        all_images = glob.glob(idir + '/*')
        label_id = int(idir.split('/')[-1].split('.')[0])-1
        if label_id > 256:
            continue
        np.random.shuffle(all_images)
        train_paths, val_paths, test_paths = all_images[:len(all_images)/10*8], all_images[len(all_images)/10*8:len(all_images)/10*9], all_images[len(all_images)/10*9:]
        train_labels, val_labels, test_labels = [label_id]*(len(all_images)/10*8), [label_id]*(len(all_images)/10), [label_id]*(len(all_images)/10)
        
        d['train_images'].append(train_paths)
        d['val_images'].append(val_paths)
        d['test_images'].append(test_paths)
        d['train_labels'].append(train_labels)
        d['val_labels'].append(val_labels)
        d['test_labels'].append(test_labels)
    return d

all_d = list_gen()

In [70]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense


def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    softmax_layer = Dense(output_dim, activation='softmax')(vgg_out) #Create softmax layer taking input as vgg_out
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    for layer in vgg_model.layers:
        layer.trainable = False
    #Confirm the model is appropriate
    return tl_model

model = getModel(num_class)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import np_utils

def preprocessing(image_paths, labels, crop_size):
    X = np.zeros((len(image_paths), crop_size, crop_size, 3), dtype=np.uint8)
    for i,path in enumerate(image_paths):
        X[i, :] = img_to_array(load_img(path, target_size=(crop_size, crop_size)))
    y = np_utils.to_categorical(labels)
    return X, y

In [63]:
crop_size = 224
all_per_class = 2
train_paths = []
val_paths = []
test_paths = []
train_labels = []
val_labels = []
test_labels = []
for i in xrange(num_class):
    train_paths += (all_d['train_images'][i][:all_per_class])
    val_paths += (all_d['val_images'][i][:all_per_class])
    test_paths += (all_d['test_images'][i][:all_per_class])
    train_labels += (all_d['train_labels'][i][:all_per_class])
    val_labels += (all_d['val_labels'][i][:all_per_class])
    test_labels += (all_d['test_labels'][i][:all_per_class])

X_train, y_train = preprocessing(train_paths, train_labels, crop_size)
X_val, y_val = preprocessing(val_paths, val_labels, crop_size)
X_test, y_test = preprocessing(test_paths, test_labels, crop_size)

In [73]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(X_train)

In [74]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    samples_per_epoch=len(X_train), nb_epoch=10)

Epoch 1/10
 32/512 [>.............................] - ETA: 1620s - loss: 8.9099 - acc: 0.0000e+00

KeyboardInterrupt: 